This is the main tutorial notebook. It is intended to introduce the basic machinery and how it is used.

First, let's import everything we need.

In [17]:
# import packages
%matplotlib inline
import numpy as np
import math
import sys
# sys.path.append('.')  # or full path to CompSep
np.math = math  # Redirect numpy.math to the built-in math module

import denoising
import importlib
denoising = importlib.reload(denoising)
import utils
utils = importlib.reload(utils)

Here we import the data. Since this is simulation data of log column density (in units of $1/cm^3$). If the data is already preprocessed, you can just import it as is.

In [18]:
# Load the column density map from file
logN_H = np.load('Archive/Turb_3.npy')[1]  
# logN_H = utils.downsample_by_four(logN_H)
nx, ny = logN_H.shape

# Define constant dust temperature
T_d = 10  # Typical Planck dust temperature in K

# int_val = []
# for nu in nu_list:
#     int_val.append(np.mean(modified_blackbody(logN_H, T_d, nu*1e9)))
# int_val = np.array(int_val)

# Observation frequency (e.g., 353 GHz in Hz)

# Compute the mock observed intensity map in μK_CMB
nu = (217,353)
I_nu_map_μK_nu1 = utils.modified_blackbody(logN_H, T_d, nu[0]*1e9)
I_nu_map_μK_nu2 = utils.modified_blackbody(logN_H, T_d, nu[1]*1e9)
I_nu_map_μK = (I_nu_map_μK_nu1, I_nu_map_μK_nu2)

The arrays that we work with need to have dimension `(1, H, W)`.

Now we define tuples of dust which will be our ground truth $s$, white noise, $n$, and finally cmb $c$. For simplicity the CMB here is modelled as a simple Gaussian random field with a falling power law for the power spectrum.

Keep in mind that in general, you can just import your own contamination array, respecting the broadcasting.

In [19]:
# Ensure inputs have shape (1, H, W)
dust_nu1 = I_nu_map_μK_nu1[None, :, :]
dust_nu2 = I_nu_map_μK_nu2[None, :, :]

dust = (dust_nu1, dust_nu2)

# --- CMB Parameters ---
n_realizations = 100
SNR = 1
amplitude = 2.
spectral_index = -1.7

nx, ny = dust_nu1.shape[-2], dust_nu1.shape[-1]

# --- Compute noise variances ---
variance_nu1 = (np.std(dust_nu1) / SNR) ** 2
variance_nu2 = (np.std(dust_nu2) / SNR) ** 2
variance = (variance_nu1, variance_nu2)

# --- Create contamination_arr with shape (n_realizations, 2, 1, H, W) ---
contamination_arr = np.zeros((n_realizations, 2, 1, nx, ny), dtype=np.float32)

for i in range(n_realizations):
    # Shared CMB: shape (1, H, W)
    cmb_map = utils.generate_cmb_map(n_x=nx, n_y=ny, amplitude=amplitude, spectral_index=spectral_index)
    cmb_map = cmb_map.cpu().numpy()[None, :, :]

    # Independent noise: shape (1, H, W)
    noise_nu1 = np.random.normal(0, np.sqrt(variance_nu1), (1, nx, ny))
    noise_nu2 = np.random.normal(0, np.sqrt(variance_nu2), (1, nx, ny))

    # Total contamination: shape (1, H, W)
    contamination_arr[i, 0] = noise_nu1 + cmb_map
    contamination_arr[i, 1] = noise_nu2 + cmb_map

contamination_arr_nu1 = contamination_arr[:, 0]  # shape: (Mn, 1, H, W)
contamination_arr_nu2 = contamination_arr[:, 1]  # shape: (Mn, 1, H, W)

Here we create our data, given by the equation $d^\nu = s^\nu + c + n^\nu$ for each frequency band $\nu$. Notice that we are assuming that we can sample configures $c \sim P(c)$ and $n \sim P(n)$. While for this case this might be possible, in general for more complicated contaminations this may be more difficult. This is a strong assumption.

We assume that $c$ is independent of $\nu$ because its spectral energy function is that of a perfect black body, and therefore $c^\nu = B(\nu, T) c_0$, where $c_0$ is some configuration and all the frequency dependce factors out into the Planck function. Therefore, we can define units (called Kelvin CMB) where this is independent of frequency.

The tuple contains the data for two channels, that is, $(d^{\nu_1}, d^{\nu_2})$.

In [20]:
noise_nu1 = np.random.normal(0, np.sqrt(variance_nu1), dust_nu1.shape)
noise_nu2 = np.random.normal(0, np.sqrt(variance_nu2), dust_nu2.shape)

noise = (noise_nu1, noise_nu2)

cmb_map = utils.generate_cmb_map(n_x=nx, n_y=ny, amplitude=amplitude, spectral_index=spectral_index)
cmb_map = cmb_map.cpu().numpy()[None, :, :]

data_nu1 = dust_nu1 + noise_nu1 + cmb_map
data_nu2 = dust_nu2 + noise_nu2 + cmb_map

# define target
data = (data_nu1, data_nu2)

# target is 
image_target = data

# definte initial maps for optimisation
image_init = image_target

If $x$ is an image, then $\phi(x)$ denotes a set of summary statistics. For example, if can be the pixel average, and the pixel wide standard deviation. In that case, it would be $\phi(x) = ( \mu, \sigma )$. We are going to work with a different kind of coefficients, called  

In [21]:
thresholding = True
if thresholding:
    M, N, J, L = dust_nu1.shape[-2], dust_nu1.shape[-1], 7, 4
    st_calc = denoising.Scattering2d(M, N, J, L)
    st_calc.add_ref(ref=data_nu1)
    s_cov = st_calc.scattering_cov(data_nu1, use_ref=True, 
                        normalization='P00', pseudo_coef=1
                    )
    st_calc.add_ref_ab(ref_a=data_nu1, ref_b=data_nu2)
    s_cov_2fields = st_calc.scattering_cov_2fields(data_nu1, data_nu2, use_ref=True, 
                        normalization='P00'
                    )
    threshold_func = denoising.threshold_func(s_cov)
    print(threshold_func)
    threshold_func_2fields = denoising.threshold_func(s_cov_2fields, two_fields=True)

else:
    threshold_func = None
    threshold_func_2fields = None

In [22]:
#TODO: If possible, try to parallelize compute_std in order to do the computation in parallel for each image instead of sequentially. 
#TODO: Make it compute the mean in addition to the std
#TODO: Choose the indices in closure, and then pass the batch directly to the loss function, instead of the full contamination_array, and then choosing within each loss computation.
std = denoising.compute_std(image_target, contamination_arr = contamination_arr, s_cov_func = threshold_func)
std_double = denoising.compute_std_double(image_target, contamination_arr = contamination_arr, s_cov_func = threshold_func_2fields)

In [23]:
n_epochs = 3 #number of epochs
# decontaminate
for i in range(n_epochs):
    print(f'Starting epoch {i+1}')
    running_map = denoising.denoise_double(image_target, contamination_arr = contamination_arr, std = std, std_double=std_double, seed=0, print_each_step=True, steps = 25, n_batch = 25, s_cov_func=threshold_func, s_cov_func_2fields=threshold_func_2fields)

    std = denoising.compute_std(running_map, contamination_arr = contamination_arr, s_cov_func = threshold_func)
    std_double = denoising.compute_std_double(running_map, contamination_arr = contamination_arr, s_cov_func = threshold_func_2fields)

#TODO change function so that you give it a list of predefined loss functions
image_syn_nu1 = running_map[0]
image_syn_nu2 = running_map[1]

Starting epoch 1
whatup
# of estimators:  54827
whatup
whatup
whatup
whatup
whatup
whatup
whatup
whatup
whatup
whatup
whatup
whatup
whatup
whatup
whatup
whatup


KeyboardInterrupt: 

In [68]:
# Convert tuples to NumPy arrays
dust = np.stack([dust_nu1[0], dust_nu2[0]])  # Shape: (2, ...)
data = np.stack([data_nu1[0], data_nu2[0]])  # Shape: (2, ...)
image_denoised = np.stack([image_syn_nu1[0], image_syn_nu2[0]])  # Ensure it's an array

cmb = True
# Create an array of objects to preserve different shapes
results = np.array([dust, data, image_denoised])
np.save(f"nu={nu}_cmb={cmb}_threshold", results)